---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [56]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():

    df=pd.read_csv("university_towns.txt",delimiter="\t",header=None,names='a')
    states= df['a'].str.contains('t]')
    k=0
    df['b']=0
    for i in range(567):
        if states[i] == True:
            k=i
        else:
            df['b'][i]=df['a'][k]
    for i in range(567):
        if states[i] == True:
            df=df.drop([i])
    df=df.reset_index(drop=True)

    for i in range(517):
        df['b'][i]= df['b'][i][0:-6]
    df['b'] = df['b'].str.replace(r"\s*\([^()]*\)","").str.strip()
    df['a'] = df['a'].str.replace(r"\s*\([^()]*\)","").str.strip()
    x=pd.DataFrame([df['a'],df['b']])
    UT=x.T
    UT=UT.rename(columns={'b':"State",'a':"RegionName"})
    Reg= UT['RegionName'].str.contains(']')

    for i in range(517):
        if Reg[i] == True:
            loc=UT['RegionName'][i].index("[")
            UT['RegionName'][i]=UT['RegionName'][i][:loc]
    for i in range(517):
        if ('(' in UT['RegionName'][i]) == True:
            loc1=UT['RegionName'][i].index("(")
            UT['RegionName'][i]=UT['RegionName'][i][:loc1]
            UT['RegionName'][i]=UT['RegionName'][i][:-1]
    for i in range(517):
        while UT['RegionName'][i][-1] == '.':
            UT['RegionName'][i]=UT['RegionName'][i][:-1]
        while UT['RegionName'][i][-1] == ',':
            UT['RegionName'][i]=UT['RegionName'][i][:-1]
    for i in range(517):
        if (',, and' in UT['RegionName'][i]) == True:
            UT['RegionName'][i]=UT['RegionName'][i][:-6]
    for i in range(100,500):
        if ('Vernon' in UT['RegionName'][i]) == True:
            UT['RegionName'][i]=UT['RegionName'][i]+','         
    return UT
    
    
    
    return UT

In [66]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    xls=pd.read_excel('gdplev.xls')
    quat = xls.iloc[7:,4:7]
    anu = xls.iloc[7:,0:3]
    anu.rename(columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars','Unnamed: 2':'GDP in billions of chained 2009 dollars'},inplace=True)
    quat.rename(columns={'Unnamed: 4':'QuaterlyYear','Unnamed: 5':'GDP in billions of current dollars','Unnamed: 6':'GDP in billions of chained 2009 dollars'},inplace=True)
    anu.reset_index(inplace=True)
    quat.reset_index(inplace=True)
    quat['GDP Change']=quat['GDP in billions of chained 2009 dollars'].diff()
    quat
    RY=[]
    for i in range (276):
        if (quat['GDP Change'][i]<0) & (quat['GDP Change'][i+1]<0):
            RY.append(quat['QuaterlyYear'][i])

    return RY[11]

In [67]:
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    xls=pd.read_excel('gdplev.xls')
    quat = xls.iloc[7:,4:7]
    anu = xls.iloc[7:,0:3]
    anu.rename(columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars','Unnamed: 2':'GDP in billions of chained 2009 dollars'},inplace=True)
    quat.rename(columns={'Unnamed: 4':'QuaterlyYear','Unnamed: 5':'GDP in billions of current dollars','Unnamed: 6':'GDP in billions of chained 2009 dollars'},inplace=True)
    anu.reset_index(inplace=True)
    quat.reset_index(inplace=True)
    quat['GDP Change']=quat['GDP in billions of current dollars'].diff()
    quat
    EY=[]
    for i in range(247,253):
        if (quat['GDP Change'][i]>0) & (quat['GDP Change'][i+1]>0):
            EY.append(quat['QuaterlyYear'][i+1])

    return EY[0]

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    xls=pd.read_excel('gdplev.xls')
    quat = xls.iloc[7:,4:7]
    anu = xls.iloc[7:,0:3]
    anu.rename(columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars','Unnamed: 2':'GDP in billions of chained 2009 dollars'},inplace=True)
    quat.rename(columns={'Unnamed: 4':'QuaterlyYear','Unnamed: 5':'GDP in billions of current dollars','Unnamed: 6':'GDP in billions of chained 2009 dollars'},inplace=True)
    anu.reset_index(inplace=True)
    quat.reset_index(inplace=True)
    quat['GDP Change']=quat['GDP in billions of current dollars'].diff()
    t=quat.iloc[246:252]['GDP in billions of chained 2009 dollars'].argmin()

    return quat['QuaterlyYear'][t]

In [74]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    pl=pd.read_csv('City_Zhvi_AllHomes.csv')
    pl.set_index(["State","RegionName"],inplace=True)
    pl2=pl.iloc[:,49:]
    pl3=pl2.groupby(np.arange(len(pl2.columns))//3, axis=1).mean()
    ren={0:'2000q1',1:'2000q2',2:'2000q3',3:'2000q4',
         4:'2001q1',5:'2001q2',6:'2001q3',7:'2001q4',
         8:'2002q1',9:'2002q2',10:'2003q3',11:'2003q4',
         12:'2003q1',13:'2003q2',14:'2003q3',15:'2003q4',
         16:'2004q1',17:'2004q2',18:'2004q3',19:'2004q4',
         20:'2005q1',21:'2005q2',22:'2005q3',23:'2005q4',
         24:'2006q1',25:'2006q2',26:'2006q3',27:'2006q4',
         28:'2007q1',29:'2007q2',30:'2007q3',31:'2007q4',
         32:'2008q1',33:'2008q2',34:'2008q3',35:'2008q4',
         36:'2009q1',37:'2009q2',38:'2009q3',39:'2009q4',
         40:'2010q1',41:'2010q2',42:'2010q3',43:'2010q4',
         44:'2011q1',45:'2011q2',46:'2011q3',47:'2011q4',
         48:'2012q1',49:'2012q2',50:'2012q3',51:'2012q4',
         52:'2013q1',53:'2013q2',54:'2013q3',55:'2013q4',
         56:'2014q1',57:'2014q2',58:'2014q3',59:'2014q4',
         60:'2015q1',61:'2015q2',62:'2015q3',63:'2015q4',
         64:'2016q1',65:'2016q2',66:'2016q3'}
    pl3.rename(columns=ren,index=states,inplace=True)
    
    return pl3

In [76]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    HDQ=convert_housing_data_to_quarters()
    gl=get_list_of_university_towns()
    HDQ['price_ratio']=HDQ['2008q2']/HDQ[get_recession_bottom()]
    HDQ['UT']=0
    HDQ.sort_index()
    HDQ.reset_index(inplace=True) 
    UTR=pd.merge(HDQ,gl,how='inner',on=['RegionName','State'])
    NUTR= HDQ.merge(gl, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
    (statistic,pvalue)=ttest_ind(NUTR['price_ratio'],UTR['price_ratio'],nan_policy='omit')
    if pvalue < 0.01:
        difference= True
    else:
        difference= False
    
    return (difference,pvalue,"university town")